In [ ]:
!pip install influxdb

     |████████████████████████████████| 77 kB 3.3 MB/s 


In [ ]:
import os
import yaml
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

from influxdb import DataFrameClient


In [ ]:
dbhost = '34.77.9.141'
dbport = 8086
dbuser = 'root'
dbpasswd = '12345678'
dbname = 'registros_contaminacion'

client = DataFrameClient(dbhost, dbport, dbuser, dbpasswd, dbname)

df = client.query('select * from ESTACION_8')['ESTACION_8']

In [ ]:
#df = df.reset_index()

In [ ]:
df = df[["NO2", "SO2", "PM10", "PM25", "O3"]] #

In [ ]:
df

,NO2,SO2,PM10,PM25,O3
2001-01-01 00:00:00+00:00,67.0,26.0,32.0,NaN,7.0
2001-01-01 01:00:00+00:00,71.0,21.0,41.0,NaN,7.0
2001-01-01 02:00:00+00:00,73.0,22.0,50.0,NaN,7.0
2001-01-01 03:00:00+00:00,75.0,19.0,55.0,NaN,7.0
2001-01-01 04:00:00+00:00,67.0,14.0,42.0,NaN,8.0
...,...,...,...,...,...
2022-04-19 17:00:00+00:00,22.0,4.0,7.0,1.0,48.0
2022-04-19 18:00:00+00:00,22.0,4.0,8.0,3.0,51.0
2022-04-19 19:00:00+00:00,19.0,4.0,7.0,1.0,60.0
2022-04-19 20:00:00+00:00,20.0,4.0,3.0,1.0,64.0


In [ ]:
# Clean data

#df_clean = df.sort_values(by='index').reset_index(drop=True)
    
df = df.dropna() # drop NaN

In [ ]:
df

,NO2,SO2,PM10,PM25,O3
2009-12-30 08:00:00+00:00,40.0,8.0,22.0,7.0,27.0
2009-12-30 09:00:00+00:00,52.0,10.0,35.0,14.0,24.0
2009-12-30 10:00:00+00:00,50.0,10.0,19.0,12.0,28.0
2009-12-30 11:00:00+00:00,45.0,10.0,22.0,8.0,38.0
2009-12-30 12:00:00+00:00,38.0,10.0,5.0,1.0,49.0
...,...,...,...,...,...
2022-04-19 17:00:00+00:00,22.0,4.0,7.0,1.0,48.0
2022-04-19 18:00:00+00:00,22.0,4.0,8.0,3.0,51.0
2022-04-19 19:00:00+00:00,19.0,4.0,7.0,1.0,60.0
2022-04-19 20:00:00+00:00,20.0,4.0,3.0,1.0,64.0


In [ ]:
# Create features

df_features = df.copy()

df_features['Hour'] = df.index.hour
df_features['Day'] = df.index.day
df_features['Month_Of_Year'] = df.index.month
df_features['Year'] = df.index.year

In [ ]:
df_features

,NO2,SO2,PM10,PM25,O3,Hour,Day,Month_Of_Year,Year
2009-12-30 08:00:00+00:00,40.0,8.0,22.0,7.0,27.0,8,30,12,2009
2009-12-30 09:00:00+00:00,52.0,10.0,35.0,14.0,24.0,9,30,12,2009
2009-12-30 10:00:00+00:00,50.0,10.0,19.0,12.0,28.0,10,30,12,2009
2009-12-30 11:00:00+00:00,45.0,10.0,22.0,8.0,38.0,11,30,12,2009
2009-12-30 12:00:00+00:00,38.0,10.0,5.0,1.0,49.0,12,30,12,2009
...,...,...,...,...,...,...,...,...,...
2022-04-19 17:00:00+00:00,22.0,4.0,7.0,1.0,48.0,17,19,4,2022
2022-04-19 18:00:00+00:00,22.0,4.0,8.0,3.0,51.0,18,19,4,2022
2022-04-19 19:00:00+00:00,19.0,4.0,7.0,1.0,60.0,19,19,4,2022
2022-04-19 20:00:00+00:00,20.0,4.0,3.0,1.0,64.0,20,19,4,2022


In [ ]:
def split_data(df,train_frac):

    train_size = int(len(df) * train_frac)
    train_df, test_df = df[:train_size], df[train_size:]

    return train_df, test_df, train_size

train_df, test_df, train_size = split_data(df_features, 0.8) # train fraction = 0.8 / train size = num de rows de entrenamiento

In [ ]:
def rescale_data(df):
  
    """Rescale all features using MinMaxScaler() to same scale, between 0 and 1."""
    
    scaler = MinMaxScaler()
    scaler = scaler.fit(df)

    df_scaled = pd.DataFrame(
        scaler.transform(df),
        index=df.index,
        columns=df.columns)
    
    return df_scaled, scaler

In [ ]:
# rescale data

train_df, scaler = rescale_data(train_df)

test_df = pd.DataFrame(
        scaler.transform(test_df),
        index=test_df.index,
        columns=test_df.columns)

In [ ]:
train_df.shape, test_df.shape

((82082, 9), (20521, 9))

## MODEL TRAINING

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
class TimeSeriesDataset(Dataset):   
    def __init__(self, X, y, seq_len=1):
        self.X = X
        self.y = y
        self.seq_len = seq_len

    def __len__(self):
        return self.X.__len__() - self.seq_len

    def __getitem__(self, index):
        return self.X[index:index+self.seq_len], self.y[index+self.seq_len]


In [ ]:
class TSModel(nn.Module):
    def __init__(self, n_features, n_hidden=64, n_layers=2):
        super(TSModel, self).__init__()

        self.n_hidden = n_hidden
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            batch_first=True,
            num_layers=n_layers,
            dropout=0.5
        )
        self.linear = nn.Linear(n_hidden, 1)
        
    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        lstm_out = hidden[-1]  # output last hidden state output
        y_pred = self.linear(lstm_out)
        
        return y_pred

In [ ]:
def train_model(
        train_df,
        test_df,
        label_name,
        sequence_length,
        batch_size,
        n_epochs,
        n_epochs_stop
):
    """Train LSTM model."""
    print("Starting with model training...")

    # create dataloaders
    train_dataset = TimeSeriesDataset(np.array(train_df), np.array(train_df[label_name]), seq_len=sequence_length)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    test_dataset = TimeSeriesDataset(np.array(test_df), np.array(test_df[label_name]), seq_len=sequence_length)
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

    # set up training
    n_features = train_df.shape[1]
    model = TSModel(n_features)
    criterion = torch.nn.MSELoss()  # L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_hist = []
    test_hist = []

    # start training
    best_loss = np.inf
    epochs_no_improve = 0
    for epoch in range(1, n_epochs+1):
        running_loss = 0
        model.train()

        for batch_idx, (data, target) in enumerate(train_loader, 1):
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            data = torch.Tensor(np.array(data))
            output = model(data)
            loss = criterion(output.flatten(), target.type_as(output))
            # if type(criterion) == torch.nn.modules.loss.MSELoss:
            #     loss = torch.sqrt(loss)  # RMSE
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        running_loss /= len(train_loader)
        train_hist.append(running_loss)

        # test loss
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for data, target in test_loader:
                data = torch.Tensor(np.array(data))
                output = model(data)
                loss = criterion(output.flatten(), target.type_as(output))
                test_loss += loss.item()
            test_loss /= len(test_loader)
            test_hist.append(test_loss)

            # early stopping
            if test_loss < best_loss:
                best_loss = test_loss

                saved_model = model.state_dict()
                #torch.save(model.state_dict(), Path(model_dir, 'model.pt'))
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
            if epochs_no_improve == n_epochs_stop:
                print("Early stopping.")
                break

        print(f'Epoch {epoch} train loss: {round(running_loss,4)} test loss: {round(test_loss,4)}')

        hist = pd.DataFrame()
        hist['training_loss'] = train_hist
        hist['test_loss'] = test_hist

    print("Completed.")

    return hist, saved_model

In [ ]:
sequence_length = 72 # 72 horas de la secuencia
batch_size = 32 # puede ser 64, 32
n_epochs = 30
n_epochs_stop = 5
label_name = 'SO2'

In [ ]:
hist, model = train_model(train_df, test_df, label_name, sequence_length, batch_size, n_epochs, n_epochs_stop)

Starting with model training...
Epoch 1 train loss: 0.003 test loss: 0.0019
Epoch 2 train loss: 0.0017 test loss: 0.002
Epoch 3 train loss: 0.001 test loss: 0.0017
Epoch 4 train loss: 0.0008 test loss: 0.0015
Epoch 5 train loss: 0.0007 test loss: 0.0012
Epoch 6 train loss: 0.0007 test loss: 0.001
Epoch 7 train loss: 0.0007 test loss: 0.0009
Epoch 8 train loss: 0.0007 test loss: 0.0008
Epoch 9 train loss: 0.0007 test loss: 0.0007
Epoch 10 train loss: 0.0007 test loss: 0.0006
Epoch 11 train loss: 0.0007 test loss: 0.0006
Epoch 12 train loss: 0.0007 test loss: 0.0006
Epoch 13 train loss: 0.0006 test loss: 0.0005
Epoch 14 train loss: 0.0006 test loss: 0.0005
Epoch 15 train loss: 0.0006 test loss: 0.0004
Epoch 16 train loss: 0.0006 test loss: 0.0004
Epoch 17 train loss: 0.0006 test loss: 0.0004
Epoch 18 train loss: 0.0006 test loss: 0.0004
Epoch 19 train loss: 0.0006 test loss: 0.0004
Epoch 20 train loss: 0.0006 test loss: 0.0003
Epoch 21 train loss: 0.0006 test loss: 0.0003
Epoch 22 train 

## EVALUATE MODEL

In [ ]:

from sklearn import metrics

def descale(
        descaler,
        values
):
    values_2d = np.array(values)[:, np.newaxis]
    return descaler.inverse_transform(values_2d).flatten()

In [ ]:
def predict(
        df,
        label_name,
        sequence_length,
        saved_model,
        saved_scaler
):
    """Make predictions."""
    
    model = TSModel(df.shape[1])
    model.load_state_dict(saved_model)
    model.eval()
    
    test_dataset = TimeSeriesDataset(np.array(df), np.array(df[label_name]), seq_len=sequence_length)
    
    
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    predictions = []
    labels = []
    with torch.no_grad():
        for features, target in test_loader:
            features = torch.Tensor(np.array(features))
            output = model(features)
            predictions.append(output.item())
            labels.append(target.item())

    # bring predictions back to original scale
    scaler = saved_scaler
    descaler = MinMaxScaler()
    descaler.min_, descaler.scale_ = scaler.min_[0], scaler.scale_[0]
    predictions_descaled = descale(descaler, predictions)
    labels_descaled = descale(descaler, labels)

    return predictions_descaled, labels_descaled

def print_loss_metrics(
        y_true,
        y_pred,
):
    print('RMSE: ', round(np.sqrt(metrics.mean_squared_error(y_true, y_pred)), 2))
    print('MAE: ', round(metrics.mean_absolute_error(y_true, y_pred), 2))

    return None

In [ ]:
predictions_descaled, labels_descaled = predict(df=test_df,
                                                          label_name=label_name,
                                                          sequence_length=sequence_length, saved_model = model, saved_scaler = scaler)

predictions_descaled.shape, predictions_descaled.shape

((20449,), (20449,))

In [ ]:
labels_descaled.shape

(20449,)

In [ ]:
print('Error on all test data:')
print_loss_metrics(labels_descaled, predictions_descaled)
print('\n')
print('Error on partial test data:')
#print_loss_metrics(labels_descaled[:10], predictions_descaled[:10])

print_loss_metrics(labels_descaled[:10], predictions_descaled[:10])

# 4.46 MAE ON ALL TEST DATA : NO2
# 4.46 MAE ON ALL TEST DATA : NO2

Error on all test data:
RMSE:  5.72
MAE:  4.33


Error on partial test data:
RMSE:  1.3
MAE:  1.27


In [ ]:
test_df[-58:-8]

,NO2,SO2,PM10,PM25,O3,Hour,Day,Month_Of_Year,Year
2022-04-06 09:00:00+00:00,0.164384,0.050847,0.040984,0.046512,0.155779,0.391304,0.166667,0.272727,1.3
2022-04-06 10:00:00+00:00,0.128767,0.050847,0.049180,0.055814,0.226131,0.434783,0.166667,0.272727,1.3
2022-04-06 11:00:00+00:00,0.090411,0.050847,0.057377,0.065116,0.281407,0.478261,0.166667,0.272727,1.3
2022-04-06 12:00:00+00:00,0.063014,0.050847,0.057377,0.069767,0.321608,0.521739,0.166667,0.272727,1.3
2022-04-06 13:00:00+00:00,0.073973,0.050847,0.076503,0.083721,0.316583,0.565217,0.166667,0.272727,1.3
2022-04-06 14:00:00+00:00,0.052055,0.050847,0.057377,0.065116,0.396985,0.608696,0.166667,0.272727,1.3
2022-04-06 15:00:00+00:00,0.052055,0.050847,0.032787,0.037209,0.407035,0.652174,0.166667,0.272727,1.3
2022-04-06 16:00:00+00:00,0.054795,0.050847,0.043716,0.055814,0.412060,0.695652,0.166667,0.272727,1.3
2022-04-06 17:00:00+00:00,0.054795,0.050847,0.030055,0.027907,0.412060,0.739130,0.166667,0.272727,1.3
2022-04-06 18:00:00+00:00,0.082192,0.050847,0.046448,0.046512,0.366834,0.782609,0.166667,0.272727,1.3


In [ ]:
torch.save(model, 'modelSO2.pt')

In [ ]:
print(labels_descaled[:10])

[22.55932203 22.55932203 22.55932203 22.55932203 22.55932203 22.55932203
 22.55932203 22.55932203 22.55932203 22.55932203]


In [ ]:
prueba_df = df.copy()

In [ ]:
prueba_df = prueba_df[:-80] # Elimina 80

prueba_df = prueba_df[-80:] # se queda con los últimos 80

prueba_df['NO2'][72:] = np.nan
prueba_df['PM10'][72:] = np.nan
prueba_df['PM25'][72:] = np.nan
prueba_df['SO2'][72:] = np.nan
prueba_df['O3'][72:] = np.nan


df_features_prueba = prueba_df.copy()

df_features_prueba['Hour'] = prueba_df.index.hour
df_features_prueba['Day'] = prueba_df.index.day
df_features_prueba['Month_Of_Year'] = prueba_df.index.month
df_features_prueba['Year'] = prueba_df.index.year
"""

data={
    'NO2':np.nan,
    'SO2':np.nan,
    'PM10':np.nan,
     'PM25': np.nan,
      'O3':np.nan
}

df_new = pd.DataFrame(data, index='2022-04-06 16:00:00+00:00')# pd.Series([np.nan,np.nan,np.nan,np.nan,np.nan])
prueba_df.append(df_new)
prueba_df




"""

df_features_prueba

,NO2,SO2,PM10,PM25,O3,Hour,Day,Month_Of_Year,Year
2022-03-31 06:00:00+00:00,35.0,4.0,7.0,5.0,16.0,6,31,3,2022
2022-03-31 07:00:00+00:00,58.0,4.0,7.0,4.0,7.0,7,31,3,2022
2022-03-31 08:00:00+00:00,68.0,4.0,17.0,12.0,4.0,8,31,3,2022
2022-03-31 09:00:00+00:00,69.0,4.0,10.0,4.0,17.0,9,31,3,2022
2022-03-31 10:00:00+00:00,50.0,4.0,23.0,13.0,32.0,10,31,3,2022
...,...,...,...,...,...,...,...,...,...
2022-04-06 11:00:00+00:00,NaN,NaN,NaN,NaN,NaN,11,6,4,2022
2022-04-06 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,12,6,4,2022
2022-04-06 13:00:00+00:00,NaN,NaN,NaN,NaN,NaN,13,6,4,2022
2022-04-06 14:00:00+00:00,NaN,NaN,NaN,NaN,NaN,14,6,4,2022


In [ ]:
df[:-80][-80:][-10:]

,NO2,SO2,PM10,PM25,O3
2022-04-06 06:00:00+00:00,18.0,4.0,4.0,3.0,72.0
2022-04-06 07:00:00+00:00,41.0,4.0,1.0,1.0,51.0
2022-04-06 08:00:00+00:00,55.0,4.0,10.0,6.0,34.0
2022-04-06 09:00:00+00:00,64.0,4.0,16.0,10.0,31.0
2022-04-06 10:00:00+00:00,51.0,4.0,19.0,12.0,45.0
2022-04-06 11:00:00+00:00,37.0,4.0,22.0,14.0,56.0
2022-04-06 12:00:00+00:00,27.0,4.0,22.0,15.0,64.0
2022-04-06 13:00:00+00:00,31.0,4.0,29.0,18.0,63.0
2022-04-06 14:00:00+00:00,23.0,4.0,22.0,14.0,79.0
2022-04-06 15:00:00+00:00,23.0,4.0,13.0,8.0,81.0


In [ ]:
df_features_prueba[-10:]

,NO2,SO2,PM10,PM25,O3,Hour,Day,Month_Of_Year,Year
2022-04-06 06:00:00+00:00,18.000000,4.000000,4.000000,3.000000,72.000000,6,6,4,2022
2022-04-06 07:00:00+00:00,41.000000,4.000000,1.000000,1.000000,51.000000,7,6,4,2022
2022-04-06 08:00:00+00:00,55.971055,21.490883,12.828208,10.875291,73.919313,8,6,4,2022
2022-04-06 09:00:00+00:00,50.504719,75.712967,19.746696,19.013238,162.362633,9,6,4,2022
2022-04-06 10:00:00+00:00,42.897647,145.085488,28.199436,29.998507,273.599927,10,6,4,2022
2022-04-06 11:00:00+00:00,41.141666,186.401904,39.973741,63.898411,332.731646,11,6,4,2022
2022-04-06 12:00:00+00:00,35.901632,226.317730,38.900739,132.323859,348.868759,12,6,4,2022
2022-04-06 13:00:00+00:00,NaN,NaN,NaN,NaN,NaN,13,6,4,2022
2022-04-06 14:00:00+00:00,NaN,NaN,NaN,NaN,NaN,14,6,4,2022
2022-04-06 15:00:00+00:00,NaN,NaN,NaN,NaN,NaN,15,6,4,2022


In [ ]:
#df_features_prueba[71:]

#df_features_prueba.iloc[72]
df_features_prueba.iloc[72]

NO2                 NaN
SO2                 NaN
PM10                NaN
PM25                NaN
O3                  NaN
Hour                8.0
Day                 6.0
Month_Of_Year       4.0
Year             2022.0
Name: 2022-04-06 08:00:00+00:00, dtype: float64

In [ ]:
modelNO2 = torch.load('modelNO2.pt')
modelO3= torch.load('modelO3.pt')
modelPM10 = torch.load('modelPM10.pt')
modelPM25 = torch.load('modelPM25.pt')
modelSO2 = torch.load('modelSO2.pt')

In [ ]:
"""
normal to scaled

test_df = pd.DataFrame(
        scaler.transform(test_df),
        index=test_df.index,
        columns=test_df.columns)


scaled to normal


scaler = saved_scaler

descaler = MinMaxScaler()
descaler.min_, descaler.scale_ = scaler.min_[0], scaler.scale_[0]
predictions_descaled = descale(descaler, predictions)
labels_descaled = descale(descaler, labels)

"""

In [ ]:
prueba_test_df = pd.DataFrame(
        scaler.transform(df_features_prueba),
        index=df_features_prueba.index,
        columns=df_features_prueba.columns)

predictions_descaledNO2, labels_descaledNO2 = predict(df=prueba_test_df,
                                                          label_name=label_name,
                                                          sequence_length=sequence_length, saved_model = modelNO2, saved_scaler = scaler)

predictions_descaledO3, labels_descaledO3 = predict(df=prueba_test_df,
                                                          label_name=label_name,
                                                          sequence_length=sequence_length, saved_model = modelO3, saved_scaler = scaler)

predictions_descaledPM10, labels_descaledPM10 = predict(df=prueba_test_df,
                                                          label_name=label_name,
                                                          sequence_length=sequence_length, saved_model = modelPM10, saved_scaler = scaler)

predictions_descaledPM25, labels_descaledPM25 = predict(df=prueba_test_df,
                                                          label_name=label_name,
                                                          sequence_length=sequence_length, saved_model = modelPM25, saved_scaler = scaler)

predictions_descaledSO2, labels_descaledSO2 = predict(df=prueba_test_df,
                                                          label_name=label_name,
                                                          sequence_length=sequence_length, saved_model = modelSO2, saved_scaler = scaler)

In [ ]:
for counter, i in enumerate(range(72,77)):

  print(i, counter)

  prueba_test_df = pd.DataFrame(
        scaler.transform(df_features_prueba),
        index=df_features_prueba.index,
        columns=df_features_prueba.columns)

  predictions_descaledNO2, labels_descaledNO2 = predict(df=prueba_test_df,
                                                            label_name=label_name,
                                                            sequence_length=sequence_length, saved_model = modelNO2, saved_scaler = scaler)

  predictions_descaledO3, labels_descaledO3 = predict(df=prueba_test_df,
                                                            label_name=label_name,
                                                            sequence_length=sequence_length, saved_model = modelO3, saved_scaler = scaler)

  predictions_descaledPM10, labels_descaledPM10 = predict(df=prueba_test_df,
                                                            label_name=label_name,
                                                            sequence_length=sequence_length, saved_model = modelPM10, saved_scaler = scaler)

  predictions_descaledPM25, labels_descaledPM25 = predict(df=prueba_test_df,
                                                            label_name=label_name,
                                                            sequence_length=sequence_length, saved_model = modelPM25, saved_scaler = scaler)

  predictions_descaledSO2, labels_descaledSO2 = predict(df=prueba_test_df,
                                                            label_name=label_name,
                                                            sequence_length=sequence_length, saved_model = modelSO2, saved_scaler = scaler)

  df_features_prueba.iloc[i,df_features_prueba.columns.get_loc('NO2')] = predictions_descaledNO2[counter]
  df_features_prueba.iloc[i,df_features_prueba.columns.get_loc('O3')] = predictions_descaledO3[counter]
  df_features_prueba.iloc[i,df_features_prueba.columns.get_loc('PM10')] = predictions_descaledPM10[counter]
  df_features_prueba.iloc[i,df_features_prueba.columns.get_loc('PM25')] = predictions_descaledPM25[counter]
  df_features_prueba.iloc[i,df_features_prueba.columns.get_loc('SO2')] = predictions_descaledSO2[counter]

72 0
73 1
74 2
75 3
76 4


In [ ]:
prueba_df_descaled = scaler.inverse_transform(prueba_df)
prueba_df_descaled

array([[3.500e+01, 4.000e+00, 7.000e+00, 5.000e+00, 1.600e+01, 6.000e+00,
        3.100e+01, 3.000e+00, 2.022e+03],
       [5.800e+01, 4.000e+00, 7.000e+00, 4.000e+00, 7.000e+00, 7.000e+00,
        3.100e+01, 3.000e+00, 2.022e+03],
       [6.800e+01, 4.000e+00, 1.700e+01, 1.200e+01, 4.000e+00, 8.000e+00,
        3.100e+01, 3.000e+00, 2.022e+03],
       [6.900e+01, 4.000e+00, 1.000e+01, 4.000e+00, 1.700e+01, 9.000e+00,
        3.100e+01, 3.000e+00, 2.022e+03],
       [5.000e+01, 4.000e+00, 2.300e+01, 1.300e+01, 3.200e+01, 1.000e+01,
        3.100e+01, 3.000e+00, 2.022e+03],
       [4.200e+01, 4.000e+00, 1.400e+01, 6.000e+00, 4.000e+01, 1.100e+01,
        3.100e+01, 3.000e+00, 2.022e+03],
       [3.200e+01, 4.000e+00, 1.500e+01, 7.000e+00, 5.100e+01, 1.200e+01,
        3.100e+01, 3.000e+00, 2.022e+03],
       [3.900e+01, 4.000e+00, 1.400e+01, 8.000e+00, 4.400e+01, 1.300e+01,
        3.100e+01, 3.000e+00, 2.022e+03],
       [2.700e+01, 4.000e+00, 2.100e+01, 1.300e+01, 5.800e+01, 1.400e+01

In [ ]:
df[-170:-100]

,NO2,SO2,PM10,PM25,O3
2022-03-29 20:00:00+00:00,45.0,4.0,29.0,14.0,19.0
2022-03-29 21:00:00+00:00,42.0,4.0,6.0,1.0,22.0
2022-03-29 22:00:00+00:00,30.0,4.0,9.0,5.0,28.0
2022-03-29 23:00:00+00:00,26.0,4.0,12.0,6.0,23.0
2022-03-30 00:00:00+00:00,19.0,4.0,18.0,9.0,26.0
...,...,...,...,...,...
2022-04-04 15:00:00+00:00,22.0,4.0,5.0,3.0,86.0
2022-04-04 16:00:00+00:00,22.0,4.0,1.0,1.0,87.0
2022-04-04 17:00:00+00:00,26.0,4.0,6.0,4.0,81.0
2022-04-04 18:00:00+00:00,35.0,4.0,4.0,3.0,72.0


In [ ]:
print('Error on all test data:')
print_loss_metrics(labels_descaled, predictions_descaled)
print('\n')
print('Error on partial test data:')
print_loss_metrics(labels_descaled[:10], predictions_descaled[:10])

Error on all test data:


ValueError: ignored